<h1 align="center">Summary of the tasks: How to run the models of the Transformers library task by task</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

# Resumo das tarefas

## Instalamos a Biblioteca Transformer

In [ ]:
# Transformers installation
%pip install transformers datasets

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

Esta página mostra os casos de uso mais frequentes ao usar a biblioteca. Os modelos disponíveis permitem diversas configurações e uma grande versatilidade nos casos de uso. As mais simples são apresentadas aqui, mostrando o uso para tarefas como `classificação de imagens` (Imagem classification), `resposta a perguntas` (Question Answering), `classificação de sequências`, `reconhecimento de entidades nomeadas` (NER) e outras.

Esses exemplos utilizam `auto-models`, que são classes que instanciarão um modelo de acordo com um determinado `checkpoint`, selecionando automaticamente a arquitetura de modelo correta. Verifique a documentação do [AutoModel](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModel) para obter mais informações. Sinta-se à vontade para modificar o código para ser mais específico e adaptá-lo ao seu caso de uso específico.

Para que um modelo tenha um bom desempenho em uma tarefa, ele deve ser carregado a partir de um checkpoint correspondente a essa tarefa. Esses pontos de verificação geralmente são `pré-treinados` em um grande corpus de dados e ajustados (`fine-tuned`) para uma tarefa específica. Isso significa o seguinte:

* Nem todos os modelos foram fine-tuned em todas as tarefas. Se você deseja fine-tune um modelo em uma tarefa específica, pode aproveitar um dos scripts `run_$TASK.py` no diretório de [exemplos](https://github.com/huggingface/transformers/tree/main/examples).


* Modelos ajustados (fine-tuned) foram ajustados em um conjunto de dados específico. Este conjunto de dados pode ou não se sobrepor ao seu caso de uso e domínio. Conforme mencionado anteriormente, você pode aproveitar os scripts de [exemplos](https://github.com/huggingface/transformers/tree/main/examples) para ajustar seu modelo ou pode criar seu próprio script de treinamento.


Para fazer uma inferência sobre uma tarefa, diversos mecanismos são disponibilizados pela biblioteca:

* `Pipelines`: abstrações muito fáceis de usar, que requerem apenas duas linhas de código.

* `Uso direto do modelo`: menos abstrações, mas mais flexibilidade e poder por meio de acesso direto a um `tokenizador` (`PyTorch/TensorFlow`) e capacidade total de inferência.


Ambas as abordagens são apresentadas aqui.

Todas as tarefas apresentadas aqui utilizam `checkpoint` pré-treinados que foram ajustados em tarefas específicas. Carregar um ponto de verificação que não foi ajustado em uma tarefa específica carregaria apenas as camadas do `Transformer` de base e não o `head adicional` que é usado para a tarefa, inicializando os pesos desse head aleatoriamente.

Isso produziria uma saída aleatória.

# Sequence Classification

A `classificação de sequências` é a tarefa de classificar as sequências de acordo com um determinado número de classes. <font color="yellow">Um exemplo de classificação de sequência</font> é o conjunto de dados `GLUE`, que é totalmente baseado nessa tarefa. Se você deseja ajustar um modelo em uma tarefa de classificação de sequência GLUE, pode aproveitar os scripts [run_glue.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py), run_tf_glue.py, run_tf_text_classification.py ou [run_xnli.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_xnli.py).

Aqui está um exemplo de uso de pipelines para fazer `análise de sentimento`: identificar se uma sequência é `positiva` ou `negativa`. Ele aproveita um modelo fine-tuned em `sst2`, que é uma tarefa `GLUE`.

Isso retorna uma Label (`"POSITIVO"` ou `"NEGATIVO"`) ao lado de uma pontuação, como segue:

In [3]:
from transformers import pipeline


classifier = pipeline("sentiment-analysis")

result = classifier("I love studying Machine Learning, that's why I became a Data Scientist.")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


label: POSITIVE, with score: 0.9982


In [4]:
result = classifier("I hate people who don't fight for their dreams.")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9642


Aqui está um exemplo de como fazer uma classificação de sequência usando um modelo para determinar se duas sequências são paráfrases uma da outra. O processo é o seguinte:

<font color="orange">1.</font> Instancie um tokenizer e um modelo a partir do nome do `checkpoint`. O modelo é identificado como um `modelo BERT` e o carrega com os pesos armazenados no checkpoint.

<font color="orange">2.</font> Construa uma sequência a partir das duas sentenças, com os separadores corretos e específicos do modelo, `ids` de tipo de token e `attention masks` (que serão criadas automaticamente pelo tokenizador).

<font color="orange">3.</font> Passe essa sequência pelo modelo para que ela seja classificada em uma das duas classes disponíveis: $0$$ (`não é uma paráfrase`) e $1$ (é uma paráfrase).

<font color="orange">4.</font> Calcule o `softmax` do resultado para obter probabilidades sobre as classes.

<font color="orange">5.</font> Imprima os resultados.

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="pt")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase).logits
not_paraphrase_classification_logits = model(**not_paraphrase).logits

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("A seguir comparamos a sequence_0 e a sequence_2, as quais SÃO PARÂFRASES! ")
# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

A seguir comparamos a sequence_0 e a sequence_2, as quais SÃO PARÂFRASES! 
not paraphrase: 10%
is paraphrase: 90%


In [12]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")


not paraphrase: 94%
is paraphrase: 6%


In [23]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

classes = ["not paraphrase", "is paraphrase"]

sequence_0 = "The company HuggingFace is based in New York City"
sequence_1 = "Apples are especially bad for your health"
sequence_2 = "HuggingFace's headquarters are situated in Manhattan"

# The tokenizer will automatically add any model specific separators (i.e. <CLS> and <SEP>) and tokens to
# the sequence, as well as compute the attention masks.
paraphrase = tokenizer(sequence_0, sequence_2, return_tensors="tf")
not_paraphrase = tokenizer(sequence_0, sequence_1, return_tensors="tf")

paraphrase_classification_logits = model(paraphrase).logits
not_paraphrase_classification_logits = model(not_paraphrase).logits

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]
not_paraphrase_results = tf.nn.softmax(not_paraphrase_classification_logits, axis=1).numpy()[0]

# Should be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(paraphrase_results[i] * 100))}%")

Some layers from the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing TFBertForSequenceClassification: ['dropout_183']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bert-base-cased-finetuned-mrpc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


not paraphrase: 10%
is paraphrase: 90%


In [24]:
# Should not be paraphrase
for i in range(len(classes)):
    print(f"{classes[i]}: {int(round(not_paraphrase_results[i] * 100))}%")

not paraphrase: 94%
is paraphrase: 6%


# Extractive Question Answering

Extrative Question Answering é a tarefa de `extrair uma resposta` de um texto a partir de uma pergunta. Um exemplo de um conjunto de dados de resposta a perguntas é o conjunto de dados `SQuAD`, que é totalmente baseado nessa tarefa. Se desejar ajustar (`fine-tune`) um modelo em uma tarefa SQuAD, você pode aproveitar os scripts [run_qa.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/question-answering/run_qa.py) e `run_tf_squad.py`.

Aqui está um exemplo de uso de `pipelines` para responder a perguntas: extrair uma resposta de um texto dado a uma pergunta. Ele aproveita um modelo `fine-tuned` no SQUAD.

In [14]:
from transformers import pipeline

question_answerer = pipeline("question-answering")


context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Isso retorna uma `resposta extraída do texto`, uma pontuação de confiança, juntamente com os valores `"start"` e `"end"`, que são as posições da resposta extraída no texto.

In [15]:
result = question_answerer(question="What is extractive question answering?", context=context)

print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)


Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95


In [16]:
result = question_answerer(question="What is a good example of a question answering dataset?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'SQuAD dataset', score: 0.5152, start: 147, end: 160


Aqui está um exemplo de resposta a perguntas (`Question Answering`) usando um modelo e um tokenizador. O processo é o seguinte:


<font color="orange">1.</font> Instancie um tokenizador e um modelo a partir do nome do checkpoint. O modelo é identificado como um `modelo BERT` e o carrega com os pesos armazenados no ponto de verificação.

<font color="orange">2.</font> Defina um texto e algumas perguntas.

<font color="orange">3.</font> Repita as perguntas e crie uma sequência a partir do texto e da pergunta atual, com os separadores corretos específicos do modelo, `IDs` de tipo de token e máscaras de atenção.

<font color="orange">4.</font> Passe esta sequência pelo modelo. Isso gera uma variedade de pontuações em todos os tokens de sequência (`pergunta e texto`), para as posições inicial e final.

<font color="orange">5.</font> Calcule o `softmax` do resultado para obter probabilidades sobre os tokens.

<font color="orange">6.</font> Busque os tokens dos valores de início (`start`) e parada (`stop`) identificados, converta esses tokens em uma string.

<font color="orange">7.</font> Imprima os resultados.

In [21]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


In [26]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]

    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]
    # Get the most likely end of answer with the argmax of the score
    answer_end = tf.argmax(answer_end_scores, axis=1).numpy()[0] + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )

    print(f"Question: {question}")
    print(f"Answer: {answer}")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch


# Language Modeling (Modelagem de linguagem)

A `modelagem de linguagem` é a tarefa de ajustar (fitting) um modelo a um corpus, que pode ser específico de um domínio. Todos os modelos baseados em Transformer populares são treinados usando uma variante de modelagem de linguagem, <font color="yellow">por exemplo:</font> `BERT` com modelagem de linguagem mascarada (Masked), `GPT-2` com modelagem de linguagem causal.

A modelagem de linguagem também pode ser útil fora do pré-treinamento, <font color="yellow">por exemplo,</font> para mudar a distribuição do modelo para um <font color="pink">domínio específico</font>: usando um modelo de linguagem treinado em um corpus muito grande e, em seguida, ajustando-o (`fine-tuning`) para um `Dataset de notícias` ou `artigos científicos`, por exemplo [LysandreJik/arxiv-nlp](https://huggingface.co/lysandre/arxiv-nlp).

## <font color="red">Masked Language Modeling (Modelagem de Linguagem Mascarada)</font>

A `modelagem de linguagem mascarada` é a tarefa de mascarar tokens em uma sequência com um token de mascaramento e solicitar que o modelo preencha essa máscara com um token apropriado. Isso permite que o modelo atenda tanto ao contexto direito (`tokens à direita da máscara`) quanto ao contexto esquerdo (`tokens à esquerda da máscara`). Esse treinamento cria uma base sólida para tarefas posteriores que exigem `contexto bidirecional`, como `SQuAD` (resposta a perguntas, consulte [Lewis, Lui, Goyal et al., parte 4.2](https://arxiv.org/abs/1910.13461)). Se desejar ajustar (`fine-tune`) um modelo em uma tarefa de **modelagem de linguagem mascarada**, você pode utilizar o script [run_mlm.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py).

Aqui está um exemplo de uso de pipelines para substituir uma máscara (`mask`) de uma sequência:

In [28]:
from transformers import pipeline

unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading:  98%|█████████▊| 1.31G/1.34G [2:10:37<02:37, 168kB/s]


Isso gera as sequências com a máscara preenchida (`Mask filled`), a pontuação de confiança e o `id` do token no vocabulário do tokenizer:

In [29]:
from pprint import pprint

pprint(
    unmasker(
        f"HuggingFace is creating a {unmasker.tokenizer.mask_token} that the community uses to solve NLP tasks."
    )
)

[{'score': 0.17927584052085876,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.11349426209926605,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.05243551358580589,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.03493541106581688,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.02860243059694767,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]


Aqui está um exemplo de `modelagem de linguagem mascarada` usando um modelo e um tokenizador. O processo é o seguinte:

<font color="yellow">1.</font> Instancie um `tokenizador` e um `modelo` a partir do nome do checkpoint. O modelo é identificado como um modelo `DistilBERT` e o carrega com os pesos armazenados no ponto de verificação.

<font color="yellow">2.</font> Defina uma sequência com um `token mascarado`, colocando o `tokenizer.mask_token` em vez de uma palavra.

<font color="yellow">3.</font> Codifique essa sequência em uma lista de IDs e encontre a posição do `token mascarado` nessa lista.

<font color="yellow">4.</font> Recupere as previsões no índice do token de máscara: ``esse tensor tem o mesmo tamanho do vocabulário`` e os valores são as pontuações atribuídas a cada token. O modelo atribui maior pontuação aos tokens que considera prováveis ​​naquele contexto.

<font color="yellow">5.</font> Recupere os 5 principais tokens usando os métodos `topk` do `PyTorch` ou `top_k` do TensorFlow.

<font color="yellow">6.</font> Substitua o token de máscara pelos tokens e imprima os resultados

In [32]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


In [34]:
from transformers import TFAutoModelForMaskedLM, AutoTokenizer
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("distilbert-base-cased")

sequence = (
    "Distilled models are smaller than the models they mimic. Using them instead of the large "
    f"versions would help {tokenizer.mask_token} our carbon footprint."
)

inputs = tokenizer(sequence, return_tensors="tf")
mask_token_index = tf.where(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]

token_logits = model(**inputs).logits
mask_token_logits = token_logits[0, mask_token_index, :]

top_5_tokens = tf.math.top_k(mask_token_logits, 5).indices.numpy()

for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Distilled models are smaller than the models they mimic. Using them instead of the large versions would help reduce our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help increase our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help decrease our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help offset our carbon footprint.
Distilled models are smaller than the models they mimic. Using them instead of the large versions would help improve our carbon footprint.


Isso imprime cinco sequências, com os $5$ principais tokens previstos pelo modelo.

## <font color="red">Causal Language Modeling (Modelagem de Linguagem Causal)</font>

A `modelagem de linguagem causal` é a tarefa de prever o token `seguindo uma sequência de tokens`. Nesta situação, o modelo atende apenas ao contexto esquerdo (`tokens à esquerda da máscara`). Tal treinamento é particularmente interessante para `tarefas de geração`. Se desejar ajustar (`fine-tune`) um modelo em uma tarefa de `modelagem de linguagem causal`, você pode aproveitar o script [run_clm.py](https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py).

Normalmente, o próximo token é previsto pela amostragem dos logits do último estado oculto que o modelo produz a partir da sequência de entrada.

Aqui está um exemplo de como usar o tokenizador e modelo e alavancar o método [top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.top_k_top_p_filtering) para amostrar o próximo token após uma sequência de entrada de tokens.

In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer, top_k_top_p_filtering
import torch
from torch import nn


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="pt")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
probs = nn.functional.softmax(filtered_next_token_logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)

generated = torch.cat([input_ids, next_token], dim=-1)

resulting_string = tokenizer.decode(generated.tolist()[0])
print(resulting_string)

Hugging Face is based in DUMBO, New York City, and features


Aqui está um exemplo de como usar o `tokenizador` e `modelo` e alavancar o método [tf_top_k_top_p_filtering()](https://huggingface.co/docs/transformers/main/en/internal/generation_utils#transformers.tf_top_k_top_p_filtering) para amostrar o próximo token após uma sequência de entrada de tokens.

In [42]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, tf_top_k_top_p_filtering
import tensorflow as tf


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = TFAutoModelForCausalLM.from_pretrained("gpt2")

sequence = f"Hugging Face is based in DUMBO, New York City, and"

inputs = tokenizer(sequence, return_tensors="tf")
input_ids = inputs["input_ids"]

# get logits of last hidden state
next_token_logits = model(**inputs).logits[:, -1, :]

# filter
filtered_next_token_logits = tf_top_k_top_p_filtering(next_token_logits, top_k=50, top_p=1.0)

# sample
next_token = tf.random.categorical(filtered_next_token_logits, dtype=tf.int32, num_samples=1)

generated = tf.concat([input_ids, next_token], axis=1)

resulting_string = tokenizer.decode(generated.numpy().tolist()[0])
print(resulting_string)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Hugging Face is based in DUMBO, New York City, and focuses


Isso gera um próximo token (espero) coerente seguindo a sequência original, que em nosso caso é a palavra `is` ou `features`.

Na próxima seção, mostramos como [generation.GenerationMixin.generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) pode ser usado para `gerar vários tokens até um comprimento especificado em vez de um token por vez`.

## <font color="red">Text Generation (Geração de texto)</font>

# Named Entity Recognition